In [1]:
!pip install pydot graphviz

In [2]:
# Importing packages
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist
from keras.utils import plot_model
import pydot
import graphviz
print("pydot:", pydot.__version__)
print("graphviz:", graphviz.__version__)

pydot: 4.0.1
graphviz: 0.20.3


### Subclassing

In [4]:
class car():
    def __init__(self,model,colour):
        self.model=model
        self.colour=colour

    def show(self):
        print(f"The model of the car is: {self.model}")
        print(f"The colour of the car is: {self.colour}")

In [5]:
audi = car("audi r8","red")
mustang = car("mustang","yellow")

In [6]:
audi.show()

The model of the car is: audi r8
The colour of the car is: red


In [7]:
mustang.show()

The model of the car is: mustang
The colour of the car is: yellow


In [8]:
class Rectange:
    def __init__(self,length,width):
        self.length =length
        self.width =width

    def area(self):
        return self.length * self.width

class Square:
    def __init__(self,length):
        self.length =length

    def area(self):
        return self.length * self.length

In [9]:
square = Square(4)
square.area()

16

In [10]:
rectange = Rectange(4,5)
rectange.area()

20

#### Inheritance

In [12]:
class Rectange:
    def __init__(self,length,width):
        self.length =length
        self.width =width

    def area(self):
        return self.length * self.width

class Square(Rectange):
     def __init__(self,length):
        super().__init__(length,length)


In [13]:
square = Square(4)
square.area()

16

In [14]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [15]:
x_train.shape,x_test.shape

((60000, 784), (10000, 784))

In [16]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.dense1 = layers.Dense(64, activation='relu')  
        self.dense2 = layers.Dense(num_classes)

    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)

    def model(self):
        x = keras.Input(shape=(28 * 28,))   
        return Model(inputs=[x], outputs=self.call(x))

In [17]:
sub_model= MyModel()

In [18]:
sub_model.model().summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
sub_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"])

In [20]:
sub_model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=1)
sub_model.evaluate(x_test,y_test,batch_size=32,verbose=1)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8698 - loss: 0.4778
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9545 - loss: 0.1549
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9684 - loss: 0.1070
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9768 - loss: 0.0818
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9799 - loss: 0.0674
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9689 - loss: 0.1092


[0.09434018284082413, 0.9732000231742859]

### Building Dense Layers

In [22]:
class custom_dense(layers.Layer):
    def __init__(self, output_dim, input_dim):
        super().__init__()
        # Initialize weights and biases
        self.w = self.add_weight(
            shape=(input_dim, output_dim),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(
            shape=(output_dim,),
            initializer="zeros",
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [23]:
class MyReLU(keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

In [24]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.dense1 = custom_dense(64, 784)  # your custom Dense
        self.dense2 = custom_dense(num_classes, 64)
        self.relu = MyReLU() 
    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)

    def model(self):
        x = keras.Input(shape=(28*28,))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [25]:
fin_model=MyModel()
fin_model.model().summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense (custom_dense)     │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_re_lu (MyReLU)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_1 (custom_dense)   │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
fin_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"])

In [27]:
fin_model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=1)
fin_model.evaluate(x_test,y_test,batch_size=32,verbose=1)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8374 - loss: 0.5866
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9491 - loss: 0.1764
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9636 - loss: 0.1242
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9726 - loss: 0.0928
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9782 - loss: 0.0740
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9662 - loss: 0.1147


[0.10070201754570007, 0.9700999855995178]

### Building Tf Datasets

In [29]:
(x_train,y_train),(x_test,y_test) =mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = tf.expand_dims(x_train,-1)
x_test = tf.expand_dims(x_test,-1)
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [30]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (train_dataset.shuffle(buffer_size=len(y_train)).batch(32,drop_remainder=True).prefetch(buffer_size=AUTOTUNE))

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = (test_dataset.batch(32,drop_remainder=False).prefetch(buffer_size=AUTOTUNE))

In [31]:
print(train_dataset)
print(test_dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(32,), dtype=tf.uint8, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [32]:
class custom_CNN(keras.layers.Layer):
    def __init__(self,filters,kernel_size=3):
        super().__init__()
        self.conv = layers.Conv2D(filters,kernel_size,padding="same")
        self.bn = layers.BatchNormalization()

    def call(self,input_tensor,training=False):
        x = self.conv(input_tensor)
        x = self.bn(x,training=training)
        x = tf.nn.relu(x)
        return x

    def model(self):
        x =keras.Input()
        return keras.Model(inputs=[x],outputs=self.call(x))

In [33]:
cus_model =keras.Sequential(
    [keras.Input(shape=(28,28,1)),
    custom_CNN(32),
     custom_CNN(64),
     custom_CNN(128),
     layers.Flatten(),
     layers.Dense(10),
    ]
)

In [34]:
cus_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ custom_cnn (custom_CNN)         │ (None, 28, 28, 32)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_cnn_1 (custom_CNN)       │ (None, 28, 28, 64)     │        18,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_cnn_2 (custom_CNN)       │ (None, 28, 28, 128)    │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │     1,003,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,097,098 (4.19 MB)

 Trainable params: 1,096,650 (4.18 MB)

 Non-trainable params: 448 (1.75 KB)

In [35]:
cus_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"])
cus_model.fit(train_dataset,batch_size=32,epochs=5,verbose=1)
cus_model.evaluate(test_dataset,batch_size=32,verbose=1)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 331s 174ms/step - accuracy: 0.9092 - loss: 1.1550
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 328s 175ms/step - accuracy: 0.9865 - loss: 0.0476
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 329s 175ms/step - accuracy: 0.9874 - loss: 0.0389
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 327s 175ms/step - accuracy: 0.9900 - loss: 0.0314
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 335s 179ms/step - accuracy: 0.9923 - loss: 0.0234
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.9825 - loss: 0.0614


[0.047421637922525406, 0.9865999817848206]

### Backward Propogation

### Gradient Tape

In [41]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
dy_dx = g.gradient(y,x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [43]:
def f(w1,w2):
    return 3 * w1**2 + 2* w1 * w2 

In [45]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1,w2)
gradients = tape.gradient(z,[w1,w2])

In [47]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [49]:
cus_model =keras.Sequential(
    [keras.Input(shape=(28,28,1)),
    custom_CNN(32),
     custom_CNN(64),
     custom_CNN(128),
     layers.Flatten(),
     layers.Dense(10),
    ]
)

In [51]:
class Custom_fit(keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def compile(self, optimizer, loss, metric):
    super(Custom_fit, self).compile()
    self.optimizer = optimizer
    self.loss = loss
    self.metric = metric

  def train_step(self,data):
    # Unpack the data
    x, y = data
    with tf.GradientTape() as tape:
      # Compute predictions
      y_pred = self.model(x, training=True)
      # Calculating loss
      loss = self.loss(y, y_pred)
    
    # Tracking gradients
    training_vars = self.trainable_variables
    gradients = tape.gradient(loss, training_vars)

    #Update optimizer & metrics
    self.optimizer.apply_gradients(zip(gradients, training_vars))
    self.metric.update_state(y, y_pred)

    return {"Train_loss for Custom_train": loss, "Train_accuracy for Custom_train": self.metric.result()}


  def test_step(self, data):
    # Unpack the data
    x, y = data
    # Compute predictions
    y_pred = self.model(x, training=False)
    # Calculating loss
    loss = self.loss(y, y_pred)
    #Update optimizer & metrics
    self.metric.update_state(y, y_pred)
    return {"Test_loss for Custom_test": loss, "Test_accuracy for Custom_test": self.metric.result()}


In [53]:
cus_train = Custom_fit(cus_model)
cus_train.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")
)

cus_train.fit(train_dataset,batch_size=32, epochs=5)
cus_train.evaluate(test_dataset, batch_size=32)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 390s 205ms/step - Train_accuracy for Custom_train: 0.9101 - Train_loss for Custom_train: 0.2936
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 342s 183ms/step - Train_accuracy for Custom_train: 0.9794 - Train_loss for Custom_train: 0.1198
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 341s 182ms/step - Train_accuracy for Custom_train: 0.9855 - Train_loss for Custom_train: 0.0630
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 332s 177ms/step - Train_accuracy for Custom_train: 0.9890 - Train_loss for Custom_train: 0.0412
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 331s 176ms/step - Train_accuracy for Custom_train: 0.9935 - Train_loss for Custom_train: 0.0243
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - Test_accuracy for Custom_test: 0.9817 - Test_loss for Custom_test: 0.0574


[<tf.Tensor: shape=(), dtype=float32, numpy=0.9854999780654907>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3.359140100656077e-05>]